# Shikari King

## Research question/interests


## Research question/interests
How do different factors contributing to happiness, such as income, social support, freedom, and corruption, vary across different regions in the world, and how do these regional differences affect overall levels of happiness and life satisfaction?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
Raw_Data_2015 = pd.read_csv('../data/raw/Raw Data/2015.csv')
Raw_Data_2016 = pd.read_csv('../data/raw/Raw Data/2016.csv')
Raw_Data_2017 = pd.read_csv('../data/raw/Raw Data/2017.csv')
Raw_Data_2018 = pd.read_csv('../data/raw/Raw Data/2018.csv')
Raw_Data_2019 = pd.read_csv('../data/raw/Raw Data/2019.csv')
Raw_Data_2020 = pd.read_csv('../data/raw/Raw Data/2020.csv')
Raw_Data_2021 = pd.read_csv('../data/raw/Raw Data/2021.csv')
Raw_Data_2022 = pd.read_csv('../data/raw/Raw Data/2022.csv')